In [1]:
import numpy as np
import pyscf

In [2]:
from pyscf import gto
mol=gto.Mole()
mol.atom='''
    O
    H 1 1.1
    H 1 1.1 2 104
'''#Z-matrix input format
mol.basis='cc-pvdz'
mol.symmetry = True
mol.build()

In [3]:
#Maximum SCF iterations
MAXITER=40
#Energy convergence criterion
E_conv=1.0e-8

In [4]:
#compute static one and two-electron quantities
#Overlap matrix
S = mol.intor('int1e_ovlp')
#number of basis Functions
nbf = S.shape[0]
print('Number of basis functions: %3d' % (nbf))

Number of basis functions:  24


In [5]:
#Build ERI Tensor
I = mol.intor('int2e')
print(np.shape(I))
#Build core Hamiltonian
T = mol.intor('int1e_kin')
V = mol.intor('int1e_nuc')
H = T + V
#print(H)
print(np.shape(H))
print(np.shape(V))

(24, 24, 24, 24)
(24, 24)
(24, 24)


to solve Roothan equation(make it a eigenvalue matrix equation), we need to transform the basis into an orthogonal basis

In [6]:
#symmetric orthogonalization
from scipy.linalg import fractional_matrix_power
A = fractional_matrix_power(S, -0.5)

In [22]:
#re-compute the ERI and core Hamiltonian tensors in the newly orthogonal AO basis
#a guess F=Hcore
# Transformed Fock matrix
F_p = A.dot(H).dot(A)
# Diagonalize F_p for eigenvalues & eigenvectors with NumPy
e, C_p = np.linalg.eigh(F_p)
C=A.dot(C_p)

# Grab occupied orbitals
ndocc=mol.nelectron//2
print(ndocc)
C_occ = C[:, :ndocc]

# Build density matrix from occupied orbitals(obtain an initial guess)
D = np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True)

5


In [23]:
E_nuc=mol.energy_nuc()
print(E_nuc)

8.002366485953992


In [24]:
#SCF Iterations
# Pre-iteration energy declarations
SCF_E = 0.0
E_old = 0.0

#this is for DIIS
# Trial & Residual Vector Lists
F_list = []
DIIS_RESID = []
D_conv = 1.0e-3
print('==> Starting SCF Iterations <==\n')

# Begin Iterations
for scf_iter in range(1, MAXITER + 1):
    # Build Fock matrix
    J = np.einsum('pqrs,rs->pq', I, D, optimize=True)
    K = np.einsum('prqs,rs->pq', I, D, optimize=True)
    F = H + 2*J - K
    
    # Build DIIS Residual(orbital gradiant after normalization)
    diis_r = A.dot(F.dot(D).dot(S) - S.dot(D).dot(F)).dot(A)
    # Append trial & residual vectors to lists
    F_list.append(F)
    DIIS_RESID.append(diis_r)
    print(len(F_list))
    # Compute RHF energy
    SCF_E = np.einsum('pq,pq->', (H + F), D, optimize=True) + E_nuc
    dE = SCF_E - E_old
    dRMS = np.mean(diis_r**2)**0.5
    print('SCF Iteration %3d: Energy = %4.16f dE = % 1.5E dRMS = %1.5E' % (scf_iter, SCF_E, dE, dRMS))
    
    # SCF Converged?
    if (abs(dE) < E_conv) and (dRMS < D_conv):
        break
    E_old = SCF_E
    
    if scf_iter >= 2:
        # Build B matrix
        B_dim = len(F_list) + 1
        B = np.empty((B_dim, B_dim))
        B[-1, :] = -1
        B[:, -1] = -1
        B[-1, -1] = 0
        for i in range(len(F_list)):
            for j in range(len(F_list)):
                B[i, j] = np.einsum('ij,ij->', DIIS_RESID[i], DIIS_RESID[j], optimize=True)

        # Build RHS of Pulay equation 
        rhs = np.zeros((B_dim))
        rhs[-1] = -1
        
        # Solve Pulay equation for c_i's with NumPy
        coeff = np.linalg.solve(B, rhs)
        
        # Build DIIS Fock matrix
        F = np.zeros_like(F)
        for x in range(coeff.shape[0] - 1):
            F += coeff[x] * F_list[x]
    
    # Compute new orbital guess with DIIS Fock matrix
    F_p =  A.dot(F).dot(A)
    e, C_p = np.linalg.eigh(F_p)
    C = A.dot(C_p)
    C_occ = C[:, :ndocc]
    D = np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True)
    
    # MAXITER exceeded?
    if (scf_iter == MAXITER):
        raise Exception("Maximum number of SCF iterations exceeded.")

# Post iterations
print('\nSCF converged.')
print('Final RHF Energy: %.8f [Eh]' % SCF_E)


==> Starting SCF Iterations <==

1
SCF Iteration   1: Energy = -68.9800327332965821 dE = -6.89800E+01 dRMS = 1.16551E-01
2
SCF Iteration   2: Energy = -69.6472544406204861 dE = -6.67222E-01 dRMS = 1.07430E-01
3
SCF Iteration   3: Energy = -75.7919291462228273 dE = -6.14467E+00 dRMS = 2.89274E-02
4
SCF Iteration   4: Energy = -75.9721892298673538 dE = -1.80260E-01 dRMS = 7.56446E-03
5
SCF Iteration   5: Energy = -75.9893690604501160 dE = -1.71798E-02 dRMS = 8.74982E-04
6
SCF Iteration   6: Energy = -75.9897163369163309 dE = -3.47276E-04 dRMS = 5.35606E-04
7
SCF Iteration   7: Energy = -75.9897932418062396 dE = -7.69049E-05 dRMS = 6.21200E-05
8
SCF Iteration   8: Energy = -75.9897956276201683 dE = -2.38581E-06 dRMS = 2.57879E-05
9
SCF Iteration   9: Energy = -75.9897957847445582 dE = -1.57124E-07 dRMS = 1.72817E-06
10
SCF Iteration  10: Energy = -75.9897957872402969 dE = -2.49574E-09 dRMS = 7.81922E-07

SCF converged.
Final RHF Energy: -75.98979579 [Eh]


In [25]:
from pyscf import scf

In [26]:
mf=scf.RHF(mol)
mf.kernel()
H_core=mf.get_hcore()
Fock=mf.get_fock()

converged SCF energy = -75.9897957874868
